# **Домашняя работа №1**

Выполнил Черницин И. А. МПМИИ231

## **Задание 1**

Необходимо написать градиентный спуск для линейной регрессии

Для реализации будем использовать MSE, проверим реализацию и сравним с sklearn реализацией

In [ ]:
import numpy as np

In [ ]:
class GDLR:
    '''
    Linear regression model with gradient descent

    :param learning_rate: loss func speed
    :param max_iter: number of iteration of alrorithm
    :param loss_history: error values for iterations
    :param w: model weights
    '''
    def __init__(self, learning_rate=0.01, max_iter=500):
        self.lr = learning_rate
        self.max_iter = max_iter
        self.loss_history = {'iteration' : [], 'loss' : []}
        self.w = None
    def _calc_loss(self, X, y):
        '''
        Method for loss calculation

        :param X: feature dataset
        :param y: target dataset

        :return loss value
        '''
        y_pred = np.dot(X, self.w)
        loss = np.sum((y_pred - y) ** 2) / len(y)
        return loss
    def _calc_gradient(self, X, y):
        '''
        Method for gradient calculation

        :param X: feature dataset
        :param y: target dataset

        :return new gradient vec
        '''
        gradient = (2 / X.shape[0]) * np.dot(X.T, (np.dot(X, self.w) - y))
        return gradient
    def _update_weights(self, gradient):
        '''
        Method makes new weights with gradient and lr values

        :param gradient: new gradient vec
        '''
        self.w -= self.lr * gradient
    def fit(self, X, y):
        '''
        Method for fit LR model using GD

        :param X: feature dataset
        :param y: target dataset

        :return None
        '''
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        for iteration in range(self.max_iter):
            gradient = self._calc_gradient(X, y)

            self._update_weights(gradient)

            loss = self._calc_loss(X, y)
            self.loss_history['iteration'].append(iteration)
            self.loss_history['loss'].append(loss)
    def predict(self, X):
        '''
        Method for make predict using model weights

        :param X: feature dataset

        :return result values array
        '''
        return np.dot(X, self.w)


In [ ]:
# Проверим на простой генерации от sklearn

In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
X, y = make_regression(n_samples=1000, n_features=5, noise=1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
%%time
model = LinearRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("MSE = ", mean_squared_error(y_test, y_pred))

MSE =  1.1090609683464694
CPU times: user 13.5 ms, sys: 1.58 ms, total: 15 ms
Wall time: 12.1 ms


In [ ]:
%%time
my_model = GDLR()

my_model.fit(X_train, y_train)
y_pred = my_model.predict(X_test)

print("MSE = ", mean_squared_error(y_test, y_pred))

MSE =  1.1111520865886781
CPU times: user 27.1 ms, sys: 0 ns, total: 27.1 ms
Wall time: 25.9 ms


In [ ]:
# В результате получили качество, сопоставимое с sklearn, времени затрачено больше, но задачи оптимизации не стояло

## Задание 2

Необходимо написать метод обратного распространения ошибки для MLP (многослойный персептрон)

Для простоты используем линейную активацию (производная = 1)

In [ ]:
class MLPRegressor:
    '''
    Regressor class using MLP and backpropagation

    :param input_size: size of input layer (number of features)
    :param hiddes_sizes: sizes of hidden layers ([layers_count, neurons_count])
    :param output_size: size of output vec
    :param learning_rate: loss func speed
    :param max_iter: number of iteration of alrorithm
    :param loss_history: error values for iterations
    :param weights: model weights
    :param biases: model biases
    '''
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01, max_iter=100):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.loss_history = {'iteration' : [], 'loss' : []}
        self.weights = []
        self.biases = []
        layer_sizes = [input_size] + hidden_sizes + [output_size]
        for i in range(len(layer_sizes)-1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]))
            self.biases.append(np.zeros(layer_sizes[i+1]))

    def _linear_activation(self, x):
        '''
        Activation func for mlp

        :param x: input value vec before activation

        :return value vec after activation
        '''
        return x

    def _compute_loss(self, y_true, y_pred):
        '''
        Method for calculating MSE

        :param y_true: real target values
        :param y_pred: predicted target values

        :return mse vec
        '''
        return np.mean((y_true - y_pred) ** 2)

    def _forward_pass(self, X):
        '''
        Method for make forward pass

        :param X: feature dataset

        :return output values
        '''
        layer_output = X
        self.layer_outputs = [layer_output]
        for i in range(len(self.weights)):
            layer_output = np.dot(layer_output, self.weights[i]) + self.biases[i]
            if i < len(self.weights) - 1:
                layer_output = self._linear_activation(layer_output)
            self.layer_outputs.append(layer_output)
        return layer_output

    def _backward_pass(self, X, y_true, y_pred):
        '''
        Method for make backward pass

        :param X: feature dataset
        :param y_true: real target values
        :param y_pred: predicted target values

        :return gradients and biases
        '''
        gradients_weights = []
        gradients_biases = []

        error = 2 * (y_pred - y_true) / y_true.shape[0]
        for i in range(len(self.weights) - 1, -1, -1):
            if i == len(self.weights) - 1:
                # Градиенты для выходного слоя
                gradients_weights.insert(0, np.dot(self.layer_outputs[i].T, error))
                gradients_biases.insert(0, np.sum(error, axis=0))
            else:
                # Градиенты для внутренних слоев
                error = np.dot(error, self.weights[i+1].T)
                gradients_weights.insert(0, np.dot(self.layer_outputs[i].T, error))
                gradients_biases.insert(0, np.sum(error, axis=0))
                error = error * 1  # Производная линейной активации = 1

        return gradients_weights, gradients_biases

    def fit(self, X, y):
        '''
        Method for model fit using forward pass and backward pass

        :param X: feature dataset
        :param y: target dataset

        :return None
        '''
        for iteration in range(self.max_iter):
            y_pred = self._forward_pass(X)

            gradients_weights, gradients_biases = self._backward_pass(X, y, y_pred)

            for i in range(len(self.weights)):
                self.weights[i] -= self.learning_rate * gradients_weights[i]
                self.biases[i] -= self.learning_rate * gradients_biases[i]

            loss = self._compute_loss(y, y_pred)
            self.loss_history['iteration'].append(iteration)
            self.loss_history['loss'].append(loss)

    def predict(self, X):
        '''
        Method for make predict using forward pass

        :param X: feature dataset

        :return result values array
        '''
        return self._forward_pass(X)

In [ ]:
X, y = make_regression(n_samples=1000, n_features=5, noise=1, random_state=42)

y = y.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

mlp_regressor = MLPRegressor(input_size=X_train.shape[1], hidden_sizes=[15], output_size=y_train.shape[1])
mlp_regressor.fit(X_train, y_train)

y_pred = mlp_regressor.predict(X_test)

print("MSE = ", mean_squared_error(y_test, y_pred))

MSE =  23.874807917208614


In [ ]:
# Получаем примерно схожие значения